In [12]:
# Some of these libraries may not be necessary...leftovers from other notebooks.

from os import listdir
from os.path import isfile, join
import fitz
import uuid

import chromadb
# from chromadb.config import Settings
# from chromadb.utils import embedding_functions
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

emb_client = OpenAI()
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return emb_client.embeddings.create(input = [text], model=model).data[0].embedding

collection_name = "EO-Simple"
cdb_path = "Chroma"
doc_path = "content"

In [7]:
# This creates the chroma collection unless it already exists...in which case, it just initializes it. 

client = chromadb.PersistentClient(path=cdb_path)

try:
    collection = client.get_collection(collection_name)
except:
    collection = client.create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})
    
item_count = collection.count()
print(f"Count of items in collection: {item_count}")

Count of items in collection: 0


In [5]:
# Here are the files...

pdffiles = [f for f in listdir(doc_path) if isfile(join(doc_path, f)) and f.lower().endswith('.pdf')]
print(str(len(pdffiles))+" PDF File(s):")
print(pdffiles)


135 PDF File(s):
['2021-01761.pdf', '2023-15347.pdf', '2022-10076.pdf', '2021-27605.pdf', '2021-19927.pdf', '2022-27585.pdf', '2021-01762.pdf', '2023-08659.pdf', '2022-13391.pdf', '2021-21908.pdf', '2021-19924.pdf', '2022-02869.pdf', '2021-09263.pdf', '2021-25548.pdf', '2023-10407.pdf', '2022-28474.pdf', '2024-04012.pdf', '2021-10691.pdf', '2021-01767.pdf', '2022-03346.pdf', '2021-02252.pdf', '2022-22834.pdf', '2023-27318.pdf', '2022-21911.pdf', '2022-11810.pdf', '2023-13889.pdf', '2024-04573.pdf', '2021-01766.pdf', '2022-20167.pdf', '2022-20210.pdf', '2021-01759.pdf', '2021-01765.pdf', '2022-05949.pdf', '2022-15743.pdf', '2021-04281.pdf', '2022-07757.pdf', '2021-27114.pdf', '2023-28661.pdf', '2021-04280.pdf', '2021-27505.pdf', '2021-02034.pdf', '2021-01924.pdf', '2021-12019.pdf', '2023-03779.pdf', '2023-16570.pdf', '2021-01852.pdf', '2023-08955.pdf', '2021-25715.pdf', '2021-07239.pdf', '2023-28662.pdf', '2021-14127.pdf', '2021-05200.pdf', '2021-01923.pdf', '2021-12382.pdf', '2022-2343

In [18]:
# Create chunks, embeddings, and vector index

chunksize = 5000

# I prevfer the LangChain recursive splitter since it's better at making logical chunks.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunksize,
    chunk_overlap  = 0,
    length_function = len,)

# Iterate through all the PDFs
for z in pdffiles:
    mydata_all = ''
    fname = join(doc_path,z)
    # The PDF text layer comes down in paragraphs so I start with building up one text layer
    doc = fitz.open(fname) 
    this_doc = ''
    for page in doc:  # iterate the document pages
        this_doc = this_doc + page.get_text() + "\n"
    mydata_all = mydata_all + this_doc + '\n'  

    # I collect the metadata going into the index
    doc_source = z
    doc_year = z.split("-")[0]
    doc_link = "https://www.federalregister.gov/d/"+z.split(".")[0]
    print(z+" contains "+str(len(mydata_all))+" characters")
    texts = text_splitter.create_documents([mydata_all])
    print("You have "+str(len(texts))+" chunks")
    
    pagecount = 0  # I like to collect the chunk number in my metadata
    # Iterate through the chunks.
    for mytext in texts:
        pagecount += 1
        myembedding = get_embedding(mytext.page_content)
        myuuid = str(uuid.uuid4()) # I like to create my own IDs though it's usually safe to default it.
        collection.add(
            embeddings=myembedding,
            metadatas={"source": doc_source,"page": pagecount, "year": doc_year, "link": doc_link},
            documents=mytext.page_content,
            ids=myuuid,
        )


2021-01761.pdf contains 6641 characters
You have 2 chunks
2023-15347.pdf contains 2041 characters
You have 1 chunks
2022-10076.pdf contains 6827 characters
You have 2 chunks
2021-27605.pdf contains 10604 characters
You have 3 chunks
2021-19927.pdf contains 6024 characters
You have 2 chunks
2022-27585.pdf contains 7810 characters
You have 2 chunks
2021-01762.pdf contains 22497 characters
You have 6 chunks
2023-08659.pdf contains 30829 characters
You have 8 chunks
2022-13391.pdf contains 26836 characters
You have 7 chunks
2021-21908.pdf contains 12396 characters
You have 4 chunks
2021-19924.pdf contains 12393 characters
You have 3 chunks
2022-02869.pdf contains 12536 characters
You have 3 chunks
2021-09263.pdf contains 14019 characters
You have 4 chunks
2021-25548.pdf contains 3752 characters
You have 1 chunks
2023-10407.pdf contains 4870 characters
You have 1 chunks
2022-28474.pdf contains 29333 characters
You have 8 chunks
2024-04012.pdf contains 12567 characters
You have 3 chunks
2021

In [36]:
# Sample Query with just Chroma...no LLM yet. 

myquery = "How should the government use AI?"

myembedding = get_embedding(myquery)

results = collection.query(
    query_embeddings=myembedding,
    n_results= 5,
#     where={"year": {"$eq": "2023"}}  # You can filter by the metadata
)

for x in range(len(results['metadatas'][0])):
    print(results['distances'][0][x])
    print("Source:",results['metadatas'][0][x]['source'])
    print("Link:",results['metadatas'][0][x]['link'])
    print("Year:",results['metadatas'][0][x]['year'])
    print("Chunks:",results['metadatas'][0][x]['page'])
    
    print("-"*40)


0.14456284046173096
Source: 2023-24283.pdf
Link: https://www.federalregister.gov/d/2023-24283
Year: 2023
Chunks: 3
----------------------------------------
0.15072143077850342
Source: 2023-24283.pdf
Link: https://www.federalregister.gov/d/2023-24283
Year: 2023
Chunks: 2
----------------------------------------
0.16301202774047852
Source: 2023-24283.pdf
Link: https://www.federalregister.gov/d/2023-24283
Year: 2023
Chunks: 33
----------------------------------------
0.16680479049682617
Source: 2023-24283.pdf
Link: https://www.federalregister.gov/d/2023-24283
Year: 2023
Chunks: 30
----------------------------------------
0.16793274879455566
Source: 2023-24283.pdf
Link: https://www.federalregister.gov/d/2023-24283
Year: 2023
Chunks: 7
----------------------------------------


In [37]:
#The most relevant block of text to your question...

print(results['documents'][0][0])

75193 
Federal Register / Vol. 88, No. 210 / Wednesday, November 1, 2023 / Presidential Documents 
especially important in critical fields like healthcare, financial services, edu-
cation, housing, law, and transportation, where mistakes by or misuse of 
AI could harm patients, cost consumers or small businesses, or jeopardize 
safety or rights. At the same time, my Administration will promote respon-
sible uses of AI that protect consumers, raise the quality of goods and 
services, lower their prices, or expand selection and availability. 
(f) Americans’ privacy and civil liberties must be protected as AI continues 
advancing. Artificial Intelligence is making it easier to extract, re-identify, 
link, infer, and act on sensitive information about people’s identities, loca-
tions, habits, and desires. Artificial Intelligence’s capabilities in these areas 
can increase the risk that personal data could be exploited and exposed. 
To combat this risk, the Federal Government will ensure th